In [ ]:
import pandas as pd
import numpy as np
import math as m
import random as rand
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
from sklearn import linear_model as lm, metrics, ensemble as ens
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import random

In [ ]:
#DEFINING A FUNCTION TO UPDATE COLUMN NAMES LATER
def lower_no_space(word): 
    
    word = re.sub(' ', '_', word) 
    
    word = re.sub(r'\'', '', word) 
    
    word = re.sub(r'\(', '', word)
    
    word = re.sub(r'\)', '', word)
    
    word = re.sub('\?', '', word)
    
    word = re.sub('/', '_', word)
    
    word = word.lower()
    
    return word

In [ ]:
df_ss = pd.read_csv("ss_gen_data.csv", header = 2)
df_mf = pd.read_csv("mf_gen_data.csv", header = 2)

In [ ]:
#RENAMING COLUMNS
df_ss = df_ss.rename(mapper = lower_no_space, axis = 1) 

#AND AGAIN
df_mf = df_mf.rename(mapper = lower_no_space, axis = 1) 

In [ ]:
#ADD OUTCOME COLUMN - ASSIGNING SS AS 1

df_ss['outcome'] = 1
df_mf['outcome'] = 0

In [ ]:
#COMBINE THEM INTO ONE "GENETIC" DATAFRAME
df_gen = pd.concat([df_ss, df_mf]) 

#NOTES:
    #SHAPE = 14577, 12
    #EACH RECORD IS A GENETIC MUTATION
    #~7K TOTAL GENES (SO ~2 RECORDS PER GENE), 143 PEOPLE
    #ONE PERSON HAS OVER 1.4K RECORDS(!!!)
    #VARIANT_CLASSIFICATION AND TYPE SEEM REDUNDANT (MUTATION = SNP; DEL = DEL; INS = INS; SPLICE SITE = NAN)


In [ ]:
df_gen.head()

In [ ]:
#CLEAN CHROMOSOME NAMES (SOME ARE JUST THE CHROM, SOME SAY 'chr' FIRST)

df_gen['chromosome'] = df_gen['chromosome'].apply(lambda x: str(x)[3:] if 'chr' in str(x) else str(x))

#STILL LEAVES 2 OBS OF 'CHROMOSOME' CHROMOSOME AND 13 OF '19_gl000209_random' CHROMOSOME; IDK WHAT TO DO WITH THESE

In [ ]:
df_gen.rename(columns={'start_position':'pos',\
                       'reference_allele' : 'ref',\
                       'altered_allele' : 'alt',\
                      'chromosome': 'chrom'}, inplace=True)

for col in ['chrom', 'pos', 'ref', 'alt']:
        df_gen = df_gen.astype({col: 'string'})

In [ ]:
chunksize = 10000000
counter = 1
        
with pd.read_csv('whole_genome_SNVs.tsv.gz', sep='\t', header = 1, chunksize = chunksize) as reader:
    for df_cadd in reader:
        print(counter, ': loaded')

        df_cadd = df_cadd.rename(mapper = lower_no_space, axis = 1) 
        df_cadd.rename(columns={'#chrom':'chrom'}, inplace=True)
        for col in ['chrom', 'pos', 'ref', 'alt']:
            df_cadd = df_cadd.astype({col: 'string'})

        df_gen = pd.merge(df_gen, df_cadd, on=['chrom', 'pos', 'ref', 'alt'], how='left')
        print(counter, ': merged')

        if 'phred_x' in df_gen.columns.values.tolist():
            df_gen['phred'] = df_gen['phred_x'].add(df_gen['phred_y'], fill_value = 0)
            df_gen['rawscore'] = df_gen['rawscore_x'].add(df_gen['rawscore_y'], fill_value = 0)

            df_gen = df_gen.drop(columns = ['phred_x', 'phred_y', 'rawscore_x', 'rawscore_y'])
        
        counter += 1


In [ ]:
df_gen.to_csv("test_gen.csv")